<a href="https://colab.research.google.com/github/Samir-atra/Other/blob/main/Capsa_experience/Capsa_Copy_of_Train%26Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports

import tensorflow as tf
import numpy as np
import os
import pathlib
import IPython
import sys
!pip install keras_tuner -q
import keras_tuner
!pip install git+https://github.com/themis-ai/capsa.git
import capsa
from capsa import EnsembleWrapper
!pip install helper
import helper

from google.colab import drive             
drive.mount('/content/drive')

In [ ]:
# Dataset loading

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')
data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')

dataset_path = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=64,
    image_size=(64, 64),
    color_mode="rgb")
    # shuffle=True)


In [ ]:
# # catch and prefetch
# AUTOTUNE=tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#classification cModel

num_classes = 1


# model = tf.keras.Sequential([
#     tf.keras.Input(shape=(64, 64, 3)),
#     # tf.keras.layers.Rescaling(1./255),
#     tf.keras.layers.Conv2D(16, 3, strides = 2, padding = "same",  activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
#     tf.keras.layers.BatchNormalization(),
#     # tf.keras.layers.Dropout(0.5),                                                                         #DP1
#     tf.keras.layers.Conv2D(32, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.BatchNormalization(),
#     # tf.keras.layers.Dropout(0.5),                                                                         #DP2
#     tf.keras.layers.Conv2D(64, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.BatchNormalization(),
#     # tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Conv2D(128, 1, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.BatchNormalization(),
#     # tf.keras.layers.Dropout(0.7),                                                                         #DP3
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units = 512, activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     # tf.keras.layers.Dropout(0.2),                                                                         #DP4                                                                        #DP5
#     tf.keras.layers.Dense(num_classes, activation= None)
# ])


# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
#     metrics=[tf.keras.metrics.BinaryAccuracy()], 
#     run_eagerly=True
# )

# model.fit(
#         dataset_path,
#         epochs=6,
#         batch_size=64,
#   )

model.summary()



In [ ]:
# the HistogramVAEWrapper check

# def build_model(hp):

model = tf.keras.Sequential([
    tf.keras.Input(shape=(64, 64, 3)),
    # tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, strides = 2, padding = "same",  activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),                                                                         #DP1
    tf.keras.layers.Conv2D(32, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),                                                                         #DP2
    tf.keras.layers.Conv2D(64, 3, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, 1, strides = 2, padding = "same", activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.7),                                                                         #DP3
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = 512, activation='relu'),#, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    # tf.keras.layers.Dropout(0.2),                                                                         #DP4                                                                        #DP5
    tf.keras.layers.Dense(num_classes, activation= None)
])


decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(units=4*4*128),  
        tf.keras.layers.Reshape((4, 4, 128)),
        tf.keras.layers.Conv2DTranspose(64, 1,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(32, 3,  strides=2, padding = "same", activation='relu'),
        tf.keras.layers.Conv2DTranspose(16, 3,  strides=2, padding = "same"),
        tf.keras.layers.Conv2DTranspose(3, 3,  strides=2, padding = "same"),
    ])


# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

wrapped_model = capsa.HistogramVAEWrapper(model, num_bins=5, queue_size=20000, latent_dim = 64, decoder=decoder)

wrapped_model.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.00001),
    loss= tf.losses.BinaryCrossentropy(from_logits= True),
    metrics= [tf.keras.metrics.BinaryAccuracy()]
)

#   return wrapped_model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="vae_compiled_binary_accuracy",
#     max_trials=5,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Tuner_Capsa",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val)

# tuner.results_summary()

# wrapped_model.summary()

history = wrapped_model.fit(
    dataset_path,
    epochs=10,                                    
    validation_data = dataset_path_val)


out = wrapped_model.predict(dataset_path_test, batch_size=512)


In [ ]:
mve_model = capsa.MVEWrapper(model, is_classification=True)

mve_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=[tf.keras.metrics.BinaryAccuracy()], 
    run_eagerly=True
)

history = mve_model.fit(
        dataset_path,
        epochs=6,
        batch_size=64,
  )

normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_train_ds = dataset_path.map(lambda x, y: (normalization_layer(x), y))
normalized_test_ds = dataset_path_test.map(lambda x, y: (normalization_layer(x), y))

images = np.vstack((normalized_test_ds, normalized_train_ds))

# print(type(images[1][0]))

# images= tf.convert_to_tensor(np.asarray(images).astype('float32'))

# predictions = model(images)

In [ ]:
Ensemble_wrapped_model = EnsembleWrapper(model,num_members=1)


Ensemble_wrapped_model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
                     )

history = Ensemble_wrapped_model.fit(
        dataset_path,
        epochs=12,
        batch_size=64,
  )

# mesh_grid = helper.get_grid()
# output = Ensemble_wrapped_model(mesh_grid)
# helper.plot_ensemble_classification(output,mesh_grid,dataset_path_test)

In [ ]:
from capsa import DropoutWrapper

base_model = unet(drop_prob=0.1)
# don't add dropout in the wrapper because our
# model already contains dropout layers
model = DropoutWrapper(base_model, p=0.0)